In [1]:
!pip install ctgan
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [2]:
from ctgan import CTGAN

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import time

# record start time
start = time.time()
le = LabelEncoder()

df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/telco_churn.csv')

df['Churn']=le.fit_transform(df['Churn'])
#transform data into numeric value
df1=df[df['Churn']==0]
df=df[df['Churn']==1]
df['Partner']=le.fit_transform(df['Partner'])
df['Dependents']=le.fit_transform(df['Dependents'])
df['PhoneService']=le.fit_transform(df['PhoneService'])
df['MultipleLines']=le.fit_transform(df['MultipleLines'])
df['InternetService']=le.fit_transform(df['InternetService'])
df['OnlineSecurity']=le.fit_transform(df['OnlineSecurity'])
df['DeviceProtection']=le.fit_transform(df['DeviceProtection'])
df['TechSupport']=le.fit_transform(df['TechSupport'])
df['StreamingTV']=le.fit_transform(df['StreamingTV'])
df['StreamingMovies']=le.fit_transform(df['StreamingMovies'])
df['Contract']=le.fit_transform(df['Contract'])
df['PaperlessBilling']=le.fit_transform(df['PaperlessBilling'])
df['PaymentMethod']=le.fit_transform(df['PaymentMethod'])
df['gender']=le.fit_transform(df['gender'])
df['OnlineBackup']=le.fit_transform(df['OnlineBackup'])

#filling missing value with mean
df['TotalCharges'].replace(to_replace = 0, value = df['TotalCharges'].mean(), inplace=True)
df=df.drop(['customerID'],axis=1)

df1['Partner']=le.fit_transform(df1['Partner'])
df1['Dependents']=le.fit_transform(df1['Dependents'])
df1['PhoneService']=le.fit_transform(df1['PhoneService'])
df1['MultipleLines']=le.fit_transform(df1['MultipleLines'])
df1['InternetService']=le.fit_transform(df1['InternetService'])
df1['OnlineSecurity']=le.fit_transform(df1['OnlineSecurity'])
df1['DeviceProtection']=le.fit_transform(df1['DeviceProtection'])
df1['TechSupport']=le.fit_transform(df1['TechSupport'])
df1['StreamingTV']=le.fit_transform(df1['StreamingTV'])
df1['StreamingMovies']=le.fit_transform(df1['StreamingMovies'])
df1['Contract']=le.fit_transform(df1['Contract'])
df1['PaperlessBilling']=le.fit_transform(df1['PaperlessBilling'])
df1['PaymentMethod']=le.fit_transform(df1['PaymentMethod'])
df1['gender']=le.fit_transform(df1['gender'])
df1['OnlineBackup']=le.fit_transform(df1['OnlineBackup'])

#filling missing value with mean
df1['TotalCharges'].replace(to_replace = 0, value = df1['TotalCharges'].mean(), inplace=True)
df1=df1.drop(['customerID'],axis=1)


batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(df)

end = time.time()



print("The time of execution of preprocess:",
      (end-start), "s")

Gen. (-0.43) | Discrim. (-0.24): 100%|██████████| 100/100 [01:21<00:00,  1.23it/s]

The time of execution of preprocess: 106.29486870765686 s


In [3]:
n_generated_data = 3305
generated_df = model.sample(n_generated_data)

#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
from sklearn.ensemble import RandomForestClassifier
import math
import time

startu=time.time()
n_generated_data = 3305
generated_df = model.sample(n_generated_data)

#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])

X=data.drop(['Churn'],axis=1)
y=data["Churn"]


#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)
cv = KFold(n_splits=5, random_state=1, shuffle=True)

endu=time.time()
print("The time of execution of preprocess undersampling:",
      (endu-startu), "s")


The time of execution of preprocess undersampling: 0.25359630584716797 s


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.797978 using {'class_weight': {0: 100, 1: 50}}
0.796070 (0.006923) with: {'class_weight': {0: 100, 1: 10}}
0.796635 (0.009095) with: {'class_weight': {0: 100, 1: 20}}
0.793876 (0.010576) with: {'class_weight': {0: 100, 1: 30}}
0.793229 (0.009284) with: {'class_weight': {0: 100, 1: 40}}
0.797978 (0.008468) with: {'class_weight': {0: 100, 1: 50}}
0.794761 (0.005375) with: {'class_weight': {0: 100, 1: 60}}
0.795970 (0.009193) with: {'class_weight': {0: 100, 1: 70}}
0.797560 (0.007022) with: {'class_weight': {0: 100, 1: 80}}
0.796781 (0.008113) with: {'class_weight': {0: 100, 1: 90}}
0.794621 (0.008784) with: {'class_weight': {0: 100, 1: 100}}


In [19]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.798396 using {'class_weight': {0: 100, 1: 80}}
0.794362 (0.006162) with: {'class_weight': {0: 100, 1: 10}}
0.795831 (0.010449) with: {'class_weight': {0: 100, 1: 20}}
0.793051 (0.010474) with: {'class_weight': {0: 100, 1: 30}}
0.790910 (0.010047) with: {'class_weight': {0: 100, 1: 40}}
0.795110 (0.009961) with: {'class_weight': {0: 100, 1: 50}}
0.793299 (0.007579) with: {'class_weight': {0: 100, 1: 60}}
0.796460 (0.009030) with: {'class_weight': {0: 100, 1: 70}}
0.798396 (0.006587) with: {'class_weight': {0: 100, 1: 80}}
0.795328 (0.009045) with: {'class_weight': {0: 100, 1: 90}}
0.793884 (0.006680) with: {'class_weight': {0: 100, 1: 100}}


In [20]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder


Best: 0.797173 using {'class_weight': {0: 100, 1: 90}}
0.793620 (0.006171) with: {'class_weight': {0: 100, 1: 10}}
0.793748 (0.011070) with: {'class_weight': {0: 100, 1: 20}}
0.793424 (0.009910) with: {'class_weight': {0: 100, 1: 30}}
0.791569 (0.010079) with: {'class_weight': {0: 100, 1: 40}}
0.795207 (0.008570) with: {'class_weight': {0: 100, 1: 50}}
0.793387 (0.005795) with: {'class_weight': {0: 100, 1: 60}}
0.796751 (0.008108) with: {'class_weight': {0: 100, 1: 70}}
0.796718 (0.007460) with: {'class_weight': {0: 100, 1: 80}}
0.797173 (0.006923) with: {'class_weight': {0: 100, 1: 90}}
0.794219 (0.008156) with: {'class_weight': {0: 100, 1: 100}}


In [21]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.898421 using {'class_weight': {0: 100, 1: 100}}
0.886942 (0.010451) with: {'class_weight': {0: 100, 1: 10}}
0.889825 (0.006898) with: {'class_weight': {0: 100, 1: 20}}
0.893559 (0.006855) with: {'class_weight': {0: 100, 1: 30}}
0.894929 (0.006602) with: {'class_weight': {0: 100, 1: 40}}
0.896917 (0.006350) with: {'class_weight': {0: 100, 1: 50}}
0.896493 (0.006104) with: {'class_weight': {0: 100, 1: 60}}
0.898139 (0.006959) with: {'class_weight': {0: 100, 1: 70}}
0.898197 (0.007073) with: {'class_weight': {0: 100, 1: 80}}
0.897654 (0.005408) with: {'class_weight': {0: 100, 1: 90}}
0.898421 (0.006776) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.808677 using {'class_weight': {0: 100, 1: 100}}
0.669384 (0.018252) with: {'class_weight': {0: 100, 1: 10}}
0.737423 (0.013632) with: {'class_weight': {0: 100, 1: 20}}
0.770985 (0.012181) with: {'class_weight': {0: 100, 1: 30}}
0.789866 (0.010401) with: {'class_weight': {0: 100, 1: 40}}
0.798368 (0.010887) with: {'class_weight': {0: 100, 1: 50}}
0.803340 (0.008415) with: {'class_weight': {0: 100, 1: 60}}
0.806891 (0.010700) with: {'class_weight': {0: 100, 1: 70}}
0.808460 (0.007901) with: {'class_weight': {0: 100, 1: 80}}
0.807596 (0.007004) with: {'class_weight': {0: 100, 1: 90}}
0.808677 (0.007651) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.808426 using {'class_weight': {0: 100, 1: 100}}
0.632687 (0.022094) with: {'class_weight': {0: 100, 1: 10}}
0.717723 (0.015762) with: {'class_weight': {0: 100, 1: 20}}
0.760104 (0.013318) with: {'class_weight': {0: 100, 1: 30}}
0.784039 (0.011347) with: {'class_weight': {0: 100, 1: 40}}
0.795311 (0.011424) with: {'class_weight': {0: 100, 1: 50}}
0.802022 (0.008742) with: {'class_weight': {0: 100, 1: 60}}
0.806482 (0.010820) with: {'class_weight': {0: 100, 1: 70}}
0.808379 (0.007949) with: {'class_weight': {0: 100, 1: 80}}
0.807525 (0.006979) with: {'class_weight': {0: 100, 1: 90}}
0.808426 (0.007558) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.894150 using {'class_weight': {0: 100, 1: 80}}
0.877902 (0.008760) with: {'class_weight': {0: 100, 1: 10}}
0.874213 (0.008017) with: {'class_weight': {0: 100, 1: 20}}
0.882271 (0.008019) with: {'class_weight': {0: 100, 1: 30}}
0.888052 (0.007572) with: {'class_weight': {0: 100, 1: 40}}
0.891116 (0.007388) with: {'class_weight': {0: 100, 1: 50}}
0.893279 (0.007150) with: {'class_weight': {0: 100, 1: 60}}
0.893952 (0.006896) with: {'class_weight': {0: 100, 1: 70}}
0.894150 (0.006789) with: {'class_weight': {0: 100, 1: 80}}
0.894125 (0.006709) with: {'class_weight': {0: 100, 1: 90}}
0.894137 (0.006606) with: {'class_weight': {0: 100, 1: 100}}


In [25]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.802203 using {'class_weight': {0: 100, 1: 80}}
0.473291 (0.009927) with: {'class_weight': {0: 100, 1: 10}}
0.695236 (0.010535) with: {'class_weight': {0: 100, 1: 20}}
0.757272 (0.011927) with: {'class_weight': {0: 100, 1: 30}}
0.779627 (0.010963) with: {'class_weight': {0: 100, 1: 40}}
0.791369 (0.009812) with: {'class_weight': {0: 100, 1: 50}}
0.797715 (0.008758) with: {'class_weight': {0: 100, 1: 60}}
0.801828 (0.009171) with: {'class_weight': {0: 100, 1: 70}}
0.802203 (0.008733) with: {'class_weight': {0: 100, 1: 80}}
0.801446 (0.008230) with: {'class_weight': {0: 100, 1: 90}}
0.801927 (0.008226) with: {'class_weight': {0: 100, 1: 100}}


In [26]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-telco1-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-tn-telco1-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.801248 using {'class_weight': {0: 100, 1: 100}}
0.375239 (0.014831) with: {'class_weight': {0: 100, 1: 10}}
0.664383 (0.012281) with: {'class_weight': {0: 100, 1: 20}}
0.740752 (0.013306) with: {'class_weight': {0: 100, 1: 30}}
0.768775 (0.011856) with: {'class_weight': {0: 100, 1: 40}}
0.784142 (0.010441) with: {'class_weight': {0: 100, 1: 50}}
0.793285 (0.009198) with: {'class_weight': {0: 100, 1: 60}}
0.798865 (0.009512) with: {'class_weight': {0: 100, 1: 70}}
0.800111 (0.009054) with: {'class_weight': {0: 100, 1: 80}}
0.800153 (0.008479) with: {'class_weight': {0: 100, 1: 90}}
0.801248 (0.008425) with: {'class_weight': {0: 100, 1: 100}}
